# Задание 1

In [1]:
#Начнём с простого - создайте Decision Tree классификатор, используя одноимённый класс из библиотеки sklearn и сохраните
#его в переменную dt.
#https://scikit-learn.org/stable/modules/tree.html - Decision Tree классификатор
#У дерева должны быть следующие параметры:
#максимальная глубина - 5 уровней
#минимальное число образцов в вершине для разделения - 5
#Подробнее узнать об имеющихся параметрах Decision Tree можно узнать в документации.
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5, min_samples_split=5)

# Задание 2

In [3]:
#Скачайте набор данных с тремя переменными: sex, exang, num. Представьте, что при помощи дерева решений мы хотим 
#классифицировать есть или нет у пациента заболевание сердца (переменная num), основываясь на двух признаках: пол (sex) и 
#наличие/отсутсвие стенокардии (exang). Обучите дерево решений на этих данных, используйте entropy в качестве критерия.

#Укажите, чему будет равняться значение Information Gain для переменной,  которая будет помещена в корень дерева.

#В ответе необходимо указать число с точностью 3 знака после запятой.

In [4]:
train_data = pd.read_csv("train_data_tree.csv")
X_train = train_data[['sex', 'exang']]
y_train = train_data.num

In [5]:
dt = DecisionTreeClassifier(criterion="entropy")
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
import seaborn as sb
from sklearn import tree
tree.plot_tree(dt, filled=True)
# 0 - индекс корня
l_node = dt.tree_.children_left[0] # индекс корня левого поддерева
n1 = dt.tree_.n_node_samples[l_node] # сэмплов в левом поддереве
e1 = dt.tree_.impurity[l_node] # энтропия в корне левого поддерева

In [7]:
dt.tree_.n_node_samples  #количество сэмплов(рассматриваемых вариантов) каждой ветки дерева

array([238, 157,  53, 104,  81,  16,  65], dtype=int64)

In [8]:
dt.tree_.impurity  #Энтропия

array([0.99587   , 0.90271134, 0.61219613, 0.97315604, 0.82562653,
       0.954434  , 0.77934984])

In [9]:
IG = 0.996 - 157*0.903/238 - 81*0.826/238 #расчет information gain
IG

0.11920588235294116

# Задание 3

In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) #генерация трейн и тест множеств
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
predicted = dt.predict(X_test)

In [11]:
predicted

array([1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 1, 0, 1, 0, 2, 2, 0, 1,
       2, 0, 1, 1, 1, 2, 0, 2, 0, 2, 2, 2, 1, 0, 2, 2])

In [12]:
#1)train_test_split Разделение имеющихся данных на тестовый и тренировочный наборы
#2)cross_validateРазделение данных на kk частей, тренировка на k - 1k−1 частях, тестирование на оставшейся; так для каждой части
#3)ShuffleSplitАналог обычного разделения на тестовый и тренировочный датасэты с большим числом таких случайных разделений
#4)LeaveOneOutРазделение данных на 2 части с n - 1n−1 и 11 наблюдением, на первой идёт тренировка, 2-ая - для предсказания; 
#каждое наблюдение побывает во второй части
#5)LeavePOutРазделение данных на 2 части с n - kn−k и kk наблюдениями, на первой идёт тренировка, 2-ая - для предсказания; 
#идёт ротация по всем наблюдениям
#6)StratifiedKFoldk-fold cross validation с учётом количества наблюдений в классах

In [13]:
#Существуют различные способы вызова кросс-валидации в sklearn, например
#cross_val_predict(estimator, x, y, cv=bar)
#где estimator - предсказывающая модель, а bar - число блоков при k-fold кросс-валидации или объект из sklearn.model_selection, 
#позволяющий осуществлять кросс-валидацию по другой стратегии.
#Мы будем использовать другой способ - GridSearchCV, отбирающий лучшую модель по заданным параметрам, проводя кросс-валидацию.

# Задание 4

In [14]:
#Одно дерево - хорошо, но где гарантии, что оно является лучшим, или хотя бы близко к нему? Одним из способов найти 
#более-менее оптимальный набор параметров дерева является перебор множества деревьев с разными параметрами и выбор подходящего.
#Для этой цели существует класс GridSearchCV, перебирающий каждое из сочетаний параметров среди заданных для модели, 
#обучающий её на данных и проводящих кросс-валидацию. После этого в аттрибуте .best_estimator_ храниться модель с 
#лучшими параметрами. Это применимо не только к деревьям, но и к другим моделям sklearn.

#Теперь задание - осуществите перебор всех деревьев на данных ириса по следующим параметрам:
#максимальная глубина - от 1 до 10 уровней
#минимальное число проб для разделения - от 2 до 10
#минимальное число проб в листе - от 1 до 10
#и сохраните в переменную best_tree лучшее дерево. Переменную с GridSearchCV назовите search

In [15]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris


iris = load_iris()
x = iris.data
y = iris.target
parameter_grid = {
            'max_depth': range(1,10),
            'min_samples_split': range(2,10),
            'min_samples_leaf':range(1,10)
    }
df = DecisionTreeClassifier()
search=GridSearchCV(df,parameter_grid)
search.fit(x, y)
best_tree = search.best_estimator_

C:\Users\Titan\Miniconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Titan\Miniconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
#Такие маны и примеры могут помочь в решении :

#https://mlbootcamp.ru/article/tutorial/

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV

#https://qa-help.ru/questions/parametr-oczenki-gridsearchcv-s-ispolzovaniem-scoring-f1-ili-scoring-none-po-umolchaniyu-ispolzuet-tochnost-daet-tot-zhe-rezultat

#https://habr.com/ru/company/ods/blog/322534/

#http://qaru.site/questions/251472/how-to-get-best-estimator-on-gridsearchcv-random-forest-classifier-scikit

#https://www.coursera.org/lecture/supervised-learning/podbor-paramietrov-po-sietkie-sklearn-grid-search-Cug3z

# Задание 5

In [17]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_iris

iris = load_iris()
x = iris.data
y = iris.target

clf = DecisionTreeClassifier(random_state=0)

clf.fit(x, y)

parameters = {'max_depth': range(1, 10),
             'min_samples_leaf': range(1, 10),
             'min_samples_split': range(2, 10)}

search = RandomizedSearchCV(clf, parameters, verbose=1, n_jobs=-1)
search.fit(x,y)
best_tree = search.best_estimator_

C:\Users\Titan\Miniconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished
C:\Users\Titan\Miniconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# Задание 6

In [18]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

parameter_grid = {
            'max_depth': range(1,10),
            'min_samples_split': range(2,10),
            'min_samples_leaf':range(1,10)
    }

x = train.drop('y',axis = 1)
y = train.y

df = DecisionTreeClassifier()
search=GridSearchCV(df,parameter_grid)  
search.fit(x, y)
best_tree = search.best_estimator_         #выбираем лучшее дерево
predictions = best_tree.predict(test) #предсказываем с помощью лучшего дерева

NameError: name 'train' is not defined

In [19]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# разбиваем датафрейм train на X и y
x_train = train.drop(["y"], axis=1)
y_train = train["y"]

# создаем словарь с параметрами, которые хотим проверить
params = {"max_depth": range(1,11), "min_samples_split":range(2,11), "min_samples_leaf":range(1,11)}

# создаем экзмпляр дерева
dt = DecisionTreeClassifier()

# создаем экземпляр GridSearchCV
search = GridSearchCV(dt, params, cv=5)

# обучаем или ищем лучшее дерево
search.fit(x_train,y_train)

# сохраням лучшее дерево по мнению GridSearchCV
best_tree = search.best_estimator_

# делаем предсказания по лучшему дереву на датасете test
predictions = best_tree.predict(test)

NameError: name 'train' is not defined

# Задание 7

In [ ]:
#При классификации модель может допускать ошибки, присваивая наблюдению неверный класс. Существуют различные метрики
#оценки качества предсказаний, которые базируются на 4-ёх параметрах - true positive, false positive, false negative и 
#true negative, соответствующих тому какой класс был присвоен наблюдениям каждого из классов. Матрицу из 4-ёх (в случае 
#бинарной классификации) этих параметров называют confusion matrix.

In [20]:
import pandas as pd
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y, predictions)
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

NameError: name 'predictions' is not defined

In [ ]:
#F 1−score Отношение удвоенного произведения precision и recall к их сумме
#RecallRecall, True Positive RateTruePositiveRate, SensitivitySensitivity Отношение правильно определённых положительных 
#случаев ко всем положительным - показывает какую часть положительных случаев модель правильно классифицирует
#PrecisionPrecision Отношение правильно определённых положительных наблюдений ко всем определённым как положительные
#AccuracyAccuracy Отношение правильно классифицированных наблюдений ко всем
#SpecificitySpecificity, True Negative RateTrueNegativeRate Отношение правильно определённых отрицательных случаев ко 
#всем отрицательным - показывает какую часть отрицательных случаев модель правильно классифицирует